# Secção de Código

In [2]:
# ATENÇÃO À UTILIZAÇÃO DESTA FUNÇÃO
# DEMASIADAS QUERIES SEGUIDAS PODEM LEVAR A BLOQUEIO POR PARTE DO SERVIDOR
# EM PRINCÍPIO O MÓDULO ENTREZ ESTÁ PREPARADO PARA LIDAR COM ISSO MAS CONVÉM TER SEMPRE CUIDADO

def pesquisa_ncbi(email, term, db = 'pubmed', retmax = 10, rettype = 'abstract', retmode = 'text', display = 'Y' , save = 'N', ext='txt'):
    '''
    NOTA - PARA USAR RETMODE XML TÊM DE SER DECLARADA UMA NOVA VARIÁVEL POIS A FUNÇÃO AINDA NÃO CRIA FICHEIROS XML
    
    Função para a pesquisa de bibliografia na base de dados NCBI utilizando BioPython
    Esta função contempla um conjunto de parâmetros para seleção de base de dados e outputs desejado adaptando-se
    ao objetivo da pesquisa. 

    Por defeito a função pesquisa a base de dados pubmed e capta os 10 primeiros resultados para uma pesquisa
    generalizada. 

    Tem a opção de gerar um ficheiro de texto com os resultados da pesquisa de artigos e respetivos abstracts.

    No entanto a função pode ser especializada para uma pesquisa em base de dados de genes para captar ficheiros
    GenBank e FASTA para posterior análise com as funções específicas que estamos a desenvolver.

    Parâmetros
    ----------

    email : str
        email a ser utilizado pelo módulo Entrez para  pesquisa (e.g. exempl@mail.com)

    term : str
        termo a ser utilizado na pesquisa (e.g. 'gene-123 functions')

    db  : str
        base de dados onde irá ser realizada a pesquisa. Por defeito é utilizada a 'pubmed'
        Para efeitos deste trabalho iremos também utilizar: 'gene', 'nuccore' e 'protein'
        Outras opções disponíveis podem ser encontradas aqui: https://www.ncbi.nlm.nih.gov/books/NBK25497/table/chapter2.T._entrez_unique_identifiers_ui/?report=objectonly

    retmax : int
        define o número de resultados que queremos retirar para o output final (10 por defeito)

    rettype : str
        define o tipo de resultado que pretendemos retirar da base de dados ('abstract' por defeito)
        pode ser mudado para 'gb' para base de dados 'gene' ou 'fasta' para base de dados de nucleotidos ou proteina (sequencias)
        
    retmode : str
        define o retmode para a captura de resultados da pesquisa na base de dados. Por defeito 
        o retmode é text mas pode ser adaptado para 'xml' quando é pretendido obter um ficheiro xml (e.g base de dados nucleotide, ou genbank)

    display : str
        define se é pretendido o output da função legível (útil para quando se quer refinar a pesquisa antes/sem gravar o ficheiro)
        por defeito apresenta o resultado em formato legível
    
    save : str
        define se é pretendido gravar ou não o ficheiro de resultados na pasta 'lit-search-output'
        por defeito este parâmetro está inactivo

            
    
    '''
    # TODO criar ficheiros XML
    


    # Importação de módulos
    from Bio import Entrez
    import os

    # Definimos o email para os fins de pesquisa
    Entrez.email = email
        
    # Secção egquery que pode ser skipable

    handle = Entrez.egquery(term=term)
    egq_res = Entrez.read(handle)

    for _ in egq_res['eGQueryResult']:
        if db in _.values():
            if retmax > 1:
                print('Encontrados {} resultados em {}. Irão ser processados {} resultados.\n'.format(_['Count'],db,retmax))
            else:
                print('Encontrados {} resultados em {}. Irá ser processado 1 resultado.\n'.format(_['Count'],db))

    
    if rettype == 'fasta' or rettype == 'gb': retmax = 1

    # Sacar as IDs
    handle      = Entrez.esearch(db=db,term=term, retmode=retmode, retmax=retmax)
    esearch_res = Entrez.read(handle)

    lista_ids = esearch_res['IdList']
    # print('Top 10 artigos ->',lista_ids) # Converter para uma lista de títulos mais à frente'''
    
    

    # Transfere a informação
    handle = Entrez.efetch(db=db,id=','.join(lista_ids),rettype=rettype,retmode=retmode)
    fetch_res = handle.read()
    
    # Usar com print para já, para não estar a bagunçar demasiado.
    # Perceber como se pode implementar a escrita de ficheiros para ser mais fácil de mastigar os resultados.

    # NÃO FUNCIONA NO GITHUB / LIVECODING
    # ADICIONAR PARAMETRO save = 'Y' SE SE PRETENDER GUARDAR FICHEIRO 
    
    if display.upper() == 'Y':
        print(fetch_res,sep='\n')

    # Verifica a condição de gravação de output como ficheiro de texto
    
    if retmode == 'xml': save = 'n'
    
    if save.upper() == 'Y':  

        # Definimos a extensão em função da base de dados e retmode
        if rettype == 'gb':
            ext = '.gbk'
        
        elif rettype == 'fasta':
            ext = '.fasta'
        
        elif retmode == 'xml':
            ext = '.xml'

        else: ext = '.txt'

        filename = term+ext

        # Verifica se existe pasta para output e cria-a caso não exista
        cwd = os.getcwd()
        outputdir = os.path.join(cwd,'gene_search() output')
        if not os.path.exists(outputdir):
            os.mkdir(outputdir)


        # Muda o working directory para a pasta de output para gerar o ficheiro
        os.chdir(outputdir)

        if os.path.isfile(filename): print('Ficheiro já existe. Não será criado novo ficheiro.')



        # Gera o ficheiro           
        with open(term+ext , 'w', encoding='utf-8') as _:
            _.write(fetch_res)

        # Retorna ao working directory anterior
        os.chdir(cwd)


    handle.close()

    return fetch_res

In [4]:
email = 'pg21019@alunos.uminho.pt'
gene = 'hla-dqa2'
termos = [
    
    'hla-dqa1 hla-dqa2 hla-dqb1',
    'hla-dqa2 food allergy',
    'hla-dqa2 cancer'
]

In [ ]:
# Pesquisa generalizada - top 10 resultaods

pesquisa_ncbi(email,gene,save='Y')

In [ ]:
# Pesquisa para os restantes termos
from time import sleep

for termo in termos:
    pesquisa_ncbi(email,termo,save='y',display='n')
    sleep(5)


In [18]:
# Identificação dos dados para pesquisa avançada
nucleotide = pesquisa_ncbi(email,gene,db='nuccore',retmax=1) # Descobrir o identificador do gene
protein_db = pesquisa_ncbi(email,gene,db='protein',save='n',retmax=1) # Descobrir o identificador da proteína

Encontrados 99 resultados em nuccore. Irá ser processado 1 resultado.

LOCUS       OR557191                5232 bp    DNA     linear   PRI 25-OCT-2023
DEFINITION  Homo sapiens isolate PSCDA0402 MHC class II antigen (HLA-DQA2)
            gene, HLA-DQA2*01:05:01:01extended allele, complete cds.
ACCESSION   OR557191
VERSION     OR557191.1
KEYWORDS    .
SOURCE      Homo sapiens (human)
  ORGANISM  Homo sapiens
            Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi;
            Mammalia; Eutheria; Euarchontoglires; Primates; Haplorrhini;
            Catarrhini; Hominidae; Homo.
REFERENCE   1  (bases 1 to 5232)
  AUTHORS   Khor,S.-S., Huijse,L., Endlich,S. and Tokunaga,K.
  TITLE     Direct Submission
  JOURNAL   Submitted (15-SEP-2023) Genome Medical Science Project, National
            Center for Global Health and Medicine, Toyama 1-21-1, Shinjuku
            Ward, Tokyo 162-8655, Japan
COMMENT     ##Assembly-Data-START##
            Assembly Method       :: NGSengi

In [45]:
nucleotide.split()

['LOCUS',
 'OR557191',
 '5232',
 'bp',
 'DNA',
 'linear',
 'PRI',
 '25-OCT-2023',
 'DEFINITION',
 'Homo',
 'sapiens',
 'isolate',
 'PSCDA0402',
 'MHC',
 'class',
 'II',
 'antigen',
 '(HLA-DQA2)',
 'gene,',
 'HLA-DQA2*01:05:01:01extended',
 'allele,',
 'complete',
 'cds.',
 'ACCESSION',
 'OR557191',
 'VERSION',
 'OR557191.1',
 'KEYWORDS',
 '.',
 'SOURCE',
 'Homo',
 'sapiens',
 '(human)',
 'ORGANISM',
 'Homo',
 'sapiens',
 'Eukaryota;',
 'Metazoa;',
 'Chordata;',
 'Craniata;',
 'Vertebrata;',
 'Euteleostomi;',
 'Mammalia;',
 'Eutheria;',
 'Euarchontoglires;',
 'Primates;',
 'Haplorrhini;',
 'Catarrhini;',
 'Hominidae;',
 'Homo.',
 'REFERENCE',
 '1',
 '(bases',
 '1',
 'to',
 '5232)',
 'AUTHORS',
 'Khor,S.-S.,',
 'Huijse,L.,',
 'Endlich,S.',
 'and',
 'Tokunaga,K.',
 'TITLE',
 'Direct',
 'Submission',
 'JOURNAL',
 'Submitted',
 '(15-SEP-2023)',
 'Genome',
 'Medical',
 'Science',
 'Project,',
 'National',
 'Center',
 'for',
 'Global',
 'Health',
 'and',
 'Medicine,',
 'Toyama',
 '1-21-1,',
 

Identificador do gene: 
Identificador da proteína: 

In [5]:
# id = 'OR557191'

# Transferimos os ficheiros GBK e FASTA para análise de features, análise de homologias, análise de proteína, etc.

pesquisa_ncbi(email,gene,db='nuccore',rettype='gb',save='Y',display='n',retmax=1)
pesquisa_ncbi(email,gene,db='nuccore',rettype='fasta',save='Y',display='n')

Encontrados 99 resultados em nuccore. Irá ser processado 1 resultado.

Encontrados 99 resultados em nuccore. Irão ser processados 10 resultados.

Ficheiro já existe. Não será criado novo ficheiro.


'>OR557191.1 Homo sapiens isolate PSCDA0402 MHC class II antigen (HLA-DQA2) gene, HLA-DQA2*01:05:01:01extended allele, complete cds\nTTCAGATTTGCTTGTCTCGAGGTCCTCACAATTGCTCTACAGCTCAGAGCAGCAACTGCTGAGGCTGCCT\nTGGGAAGAAGATGATCCTAAACAAAGCTCTGCTGCTGGGGGCCCTCGCCCTGACTGCCGTGATGAGCCCC\nTGTGGAGGTGAAGACATTGTGGGTGAGTGCATGAGTGAGGAATGTTCTCTGGAGCTGAAAAACAGCAAAT\nTAAAGGAAAAGAAAGAGTGCAATTTGCTAAGAAATAGTAGAAATTTCCCAAGGGTCTTTTCAATATTAAG\nAAATTTTAAAATTATGGCAGTTCCTCCTTTAGGAAACCAGAGCTCCAACCGACTCTCTTTGCTACCTGTG\nCTATTGGAGTTTACCAAGGACGTTGTTCTGTTTATATTATATCCAGAGACTATAGCCTGGAGGTCTGTGT\nGGCATTCCATCATGATTGCCTCAAAGACTAGGGATGTTTCCATGAATGGAGTATTTTTTTGTTATTAAAA\nATTTCTGAACTGTTACTCCCAAATTTCTCTGAACAACTTTTGAAGCTTTTCATATGCCTCCTATAGCATA\nTGTTGGGGTAGATAGTTCCATGAAGTATGTACACTCTATAGATATAAAGAAAGAGGTTCTTTTCTTTCTC\nTCAGACTTACATTTCCACATGGGAATTGGCACAGGTGGGGAGTAGGTGAAAGAGCCCAGCAGGCTGAATG\nCCTTCAACAATCATTTTACCACGTGGTAAATGTGGTACTTACTCTCTGCTACCTCATATATGTCACCTCG\nCTTATGATCAAATAAAATGGGCATGTAGATATGCTTTATGAATAGTAAAAACATGAATGTCAACTTTTTT\nTAA

In [9]:
def parsing(nome_ficheiro):

    from Bio import SeqIO
    
    record = SeqIO.read(nome_ficheiro, "genbank")

    print("ID do gene:", record.id)

    print("Nome do gene:", record.name)

    print("Descrição do gene", record.description)

    print("Comprimento da sequência:", len(record.seq), "bp")

    return

In [13]:
nome_ficheiro = nf = 'gene_search() output\\hla-dqa2.gbk'

In [10]:
parsing(nf)

ID do gene: OR557191.1
Nome do gene: OR557191
Descrição do gene Homo sapiens isolate PSCDA0402 MHC class II antigen (HLA-DQA2) gene, HLA-DQA2*01:05:01:01extended allele, complete cds
Comprimento da sequência: 5232 bp


In [11]:
def anot(nome_ficheiro):
    
    from Bio import SeqIO
    
    record = SeqIO.read(nome_ficheiro, "genbank")

    print("Quantidade de anotações:", len(record.annotations))

    print()
    
    print("Lista de anotações:")
    
    for anotacao in record.annotations:
        print(anotacao, "->", record.annotations[anotacao])

    return   


In [14]:
anot(nf)

Quantidade de anotações: 12

Lista de anotações:
molecule_type -> DNA
topology -> linear
data_file_division -> PRI
date -> 25-OCT-2023
accessions -> ['OR557191']
sequence_version -> 1
keywords -> ['']
source -> Homo sapiens (human)
organism -> Homo sapiens
taxonomy -> ['Eukaryota', 'Metazoa', 'Chordata', 'Craniata', 'Vertebrata', 'Euteleostomi', 'Mammalia', 'Eutheria', 'Euarchontoglires', 'Primates', 'Haplorrhini', 'Catarrhini', 'Hominidae', 'Homo']
references -> [Reference(title='Direct Submission', ...)]
structured_comment -> defaultdict(<class 'dict'>, {'Assembly-Data': {'Assembly Method': 'NGSengine v. 2.30.1.29498', 'Sequencing Technology': 'PacBio'}})


In [15]:
def features_qualifiers(nome_ficheiro):
    
    from Bio import SeqIO
    
    record = SeqIO.read(nome_ficheiro, "genbank")
    
    print("Quantidade de features:", len(record.features))

    for feature in record.features:
        print(feature)
    return


In [16]:
features_qualifiers(nf)

Quantidade de features: 13
type: source
location: [0:5232](+)
qualifiers:
    Key: db_xref, Value: ['taxon:9606']
    Key: isolate, Value: ['PSCDA0402']
    Key: mol_type, Value: ['genomic DNA']
    Key: organism, Value: ['Homo sapiens']

type: gene
location: [0:5232](+)
qualifiers:
    Key: allele, Value: ['HLA-DQA2*01:05:01:01extended']
    Key: gene, Value: ['HLA-DQA2']

type: mRNA
location: join{[0:162](+), [3796:4045](+), [4428:4710](+), [4877:5232](+)}
qualifiers:
    Key: allele, Value: ['HLA-DQA2*01:05:01:01extended']
    Key: gene, Value: ['HLA-DQA2']
    Key: product, Value: ['MHC class II antigen']

type: exon
location: [0:162](+)
qualifiers:
    Key: allele, Value: ['HLA-DQA2*01:05:01:01extended']
    Key: gene, Value: ['HLA-DQA2']
    Key: number, Value: ['1']

type: 5'UTR
location: [0:80](+)
qualifiers:
    Key: allele, Value: ['HLA-DQA2*01:05:01:01extended']
    Key: gene, Value: ['HLA-DQA2']

type: CDS
location: join{[80:162](+), [3796:4045](+), [4428:4710](+), [4877:50